In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from transfer_learning import TransferLearning
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

# Set parameters
data_path = "data"
batch_size = 128
lr = 1e-3

# Initialize the model
model = TransferLearning(data_path=data_path, batch_size=batch_size, lr=lr)

# Configure callbacks
early_stop_callback = EarlyStopping(monitor="val_loss", mode="min", patience=5)
checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode="min")

# Initialize the Trainer
trainer = pl.Trainer(callbacks=[early_stop_callback, checkpoint_callback], max_epochs=50, accelerator="gpu")

# Train the model
trainer.fit(model)

# Evaluate the model on the test set
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
])
test_dataset = CIFAR10(data_path, transform=test_transforms, download=True, train=False)
test_loader = DataLoader(test_dataset, batch_size=256, num_workers=8, shuffle=False)

test_results = trainer.test(model, dataloaders=test_loader)

# Print the metrics
print("Training and Validation Metrics:")
train_loss = trainer.callback_metrics.get("train_loss", None)
val_loss = trainer.callback_metrics.get("val_loss", None)
val_accuracy = trainer.callback_metrics.get("val_accuracy", None)

print(f"Training Loss: {train_loss.item() if train_loss is not None else 'N/A'}")
print(f"Validation Loss: {val_loss.item() if val_loss is not None else 'N/A'}")
print(f"Validation Accuracy: {val_accuracy.item() if val_accuracy is not None else 'N/A'}")

print("Test Metrics:")
for result in test_results:
    test_accuracy = result.get('test_accuracy', 'N/A')
    test_loss = result.get('test_loss', 'N/A')
    print(f"Test Accuracy: {test_accuracy}")
    print(f"Test Loss: {test_loss}")

# Save the model
torch.save(model.state_dict(), "cifar10_transfer_learned_model.pth")
